In [68]:
import csv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import UserDB, CancionDB, InteraccionDB
import pandas as pd
from tqdm import tqdm

# Crear una sesión de SQLAlchemy
engine = create_engine('sqlite:///database.db')
Session = sessionmaker(bind=engine)
session = Session()

# Leer el archivo CSV utilizando Pandas
df = pd.read_csv('df.csv')
users = df.drop_duplicates('userid')
songs = df.drop_duplicates('traname')

In [69]:
from sqlalchemy.exc import IntegrityError  # Asegúrate de importar IntegrityError


# Obtener el total de filas en el DataFrame
total_rows = len(users)

# Inicializar la barra de progreso
progress_bar = tqdm(total=total_rows, desc="Procesando filas")

# Iterar sobre cada fila del DataFrame
for index, row in users.iterrows():
    try:
        # Intentar crear y agregar un nuevo usuario
        user = UserDB(username=row['userid'], password=row['userid'])
        session.add(user)
        
    except IntegrityError:
        # Si hay un error de integridad, hacer rollback de la sesión para evitar abortar todo el proceso
        session.rollback()
        # Aquí podrías registrar el error o hacer algo con el registro que causó el problema
        print(f"Error al insertar usuario o canción para la fila {index}. Usuario o canción ya existen.")
    
    # Actualizar la barra de progreso independientemente de si hubo un error o no

    progress_bar.update(1)
session.commit()

# Cerrar la barra de progreso
progress_bar.close()


Procesando filas:   0%|          | 0/992 [00:00<?, ?it/s]

Procesando filas: 100%|██████████| 992/992 [00:00<00:00, 11374.45it/s]


In [70]:
from sqlalchemy.exc import IntegrityError  # Asegúrate de importar IntegrityError


# Obtener el total de filas en el DataFrame
total_rows = len(songs)

# Inicializar la barra de progreso
progress_bar = tqdm(total=total_rows, desc="Procesando filas")

# Iterar sobre cada fila del DataFrame
for index, row in songs.iterrows():
    try:
        # Intentar crear y agregar un nuevo usuario
        song = CancionDB(titulo=row['traname'])
        session.add(song)
    except IntegrityError:
        # Si hay un error de integridad, hacer rollback de la sesión para evitar abortar todo el proceso
        session.rollback()
        # Aquí podrías registrar el error o hacer algo con el registro que causó el problema
        print(f"Error al insertar usuario o canción para la fila {index}. Usuario o canción ya existen.")
    
    # Actualizar la barra de progreso independientemente de si hubo un error o no
    progress_bar.update(1)
session.commit()

# Cerrar la barra de progreso
progress_bar.close()


Procesando filas:   0%|          | 0/1083471 [00:00<?, ?it/s]

Procesando filas: 100%|██████████| 1083471/1083471 [01:35<00:00, 11350.90it/s]


In [71]:
import requests
from tqdm import tqdm

# Assuming df is your DataFrame containing interaction data
total_rows = len(df)
progress_bar = tqdm(total=total_rows, desc="Processing interactions")

# Base URL of your FastAPI application
api_url = "http://127.0.0.1:8000"

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    try:
        # Prepare the data for the API request
        user = session.query(UserDB).filter(UserDB.username == row['userid']).first()
        song = session.query(CancionDB).filter(CancionDB.titulo == row['traname']).first()
        iter = InteraccionDB(user_id=user.id, cancion_id=song.id, rating=float(row['frecuencia']), user=user, cancion=song)
        session.add(iter)
        
        progress_bar.update(1)
        
    except IntegrityError:
        # Si hay un error de integridad, hacer rollback de la sesión para evitar abortar todo el proceso
        session.rollback()
        # Aquí podrías registrar el error o hacer algo con el registro que causó el problema
        print(f"Error al insertar usuario o canción para la fila {index}. Usuario o canción ya existen.")

session.commit()
        

# Close the progress bar after processing all rows
progress_bar.close()


Processing interactions:   0%|          | 855/4407910 [00:10<15:00:47, 81.54it/s]

In [ ]:
# import requests

# # Obtener el total de filas en el DataFrame
# total_rows = len(df)
# progress_bar = tqdm(total=total_rows, desc="Procesando filas")

# # Inicializar la barra de progreso
# progress_bar = tqdm(total=total_rows, desc="Procesando interacciones")
# api_url = "http://127.0.0.1:8000"
# endpoint = f"{api_url}/interactions/"

# # Iterar sobre cada fila del DataFrame
# for index, row in df.iterrows():

#     try:
#         user = session.query(UserDB).filter(UserDB.username == row['userid']).first()
#         song = session.query(CancionDB).filter(CancionDB.titulo == row['traname']).first()

#         # Create a new InteraccionDB instance 
#         data = {
#         "user_id": user.id,
#         "cancion_id": song.id,
#         "rating": float(row['frecuencia'])
#         }
#         response = requests.post(endpoint, json=data)
        
#         progress_bar.update(1)
#     except Exception as e:
#         print(f"Error: {e}")

#     finally:
#         session.close()  # Close the session whether or not there was an error
# progress_bar.close()